# Machine Learning in Python - Roll your Own Estimator SOLUTION

## Imports Use Packages Etc

In [1]:
from IPython.display import display, HTML, Image

from TAS_Python_Utilities import data_viz
from TAS_Python_Utilities import data_viz_target
from TAS_Python_Utilities import visualize_tree

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
from random import randint

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils.multiclass import unique_labels
from sklearn import metrics
from scipy.spatial import distance


%matplotlib inline
#%qtconsole

ModuleNotFoundError: No module named 'TAS_Python_Utilities'

## Define TemplateMatchClassifier

The TemplateMatchClassifier algorithm is a simple template matching predictor. TemplateMatchClassifier only works for continuous descriptive features and categorical target features. TemplateMatchClassifier works very simply as follows:
* **Training:** For each target feature level calculate the average value of all descriptive features for instances that have that target level. Store these average vectors as templates for each target level.
* **Prediction:** When a new prediction needs to be made compare the descriptive feature values of the new query instance to each template and return the target feature level that belongs to the template that is cloesest (based on Euclidean distance) to the query case.

### Define the TemplateMatchClassifier Class

In [ ]:
# Create a new classifier which is based on the sckit-learn BaseEstimator and ClassifierMixin classes
class TemplateMatchClassifier(BaseEstimator, ClassifierMixin):
    """A very simple template match classifier. The TemplateMatchClassifier algorithm is a simple template matching predictor. TemplateMatchClassifier only works for continuous descriptive features and categorical target features. TemplateMatchClassifier works very simply as follows:
    - Training: For each target feature level calculate the average value of all descriptive features for instances that have that target level. Store these average vectors as templates for each target level.
    - Prediction: When a new prediction needs to be made compare the descriptive feature values of the new query instance to each template and return the target feature level that belongs to the template that is cloesest (based on Euclidean distance) to the query case.

    Parameters
    ----------
    distance_metric string, optional (default = 'euclidean')
        The distance metric that can be used, can be one of 'euclidean',
        'cosine', or 'manhattan'.

    Attributes
    ----------
    classes_ : array of shape = [n_classes] 
        The classes labels (single output problem).
    distance_metric: string
        The distance metric used    
    templates_: dict
        A dictionary of the templates used for each class. The classes are the dictioanry keys, and the templates the values.
        
    Notes
    -----
    

    See also
    --------
    
    ----------
    
    Examples
    --------
    >>> from sklearn.datasets import load_iris
    >>> from sklearn.model_selection import cross_val_score
    >>> clf = TemplateMatchClassifier()
    >>> iris = load_iris()
    >>> cross_val_score(clf, iris.data, iris.target, cv=10)

    """
    
    # Constructor for the classifier object
    def __init__(self, distance_metric = 'euclidean', demo_param='demo'):
        self.demo_param = demo_param
        self.distance_metric = distance_metric

    # The fit function to train a classifier
    def fit(self, X, y):
        """Build a decision tree classifier from the training set (X, y).
        Parameters
        ----------
        X : array-like or sparse matrix, shape = [n_samples, n_features]
            The training input samples. Internally, it will be converted to
            ``dtype=np.float32`` and if a sparse matrix is provided
            to a sparse ``csc_matrix``.
        y : array-like, shape = [n_samples] 
            The target values (class labels) as integers or strings.
        Returns
        -------
        self : object
        """
    
        # Convert the string distance metric into a function reference
        if self.distance_metric == 'euclidean':
            self.distance_function_ = distance.euclidean
        elif self.distance_metric == 'cosine':
            self.distance_function_ = distance.cosine
        elif self.distance_metric == 'manhattan':
            self.distance_function_ = distance.cityblock
        else:
            self.distance_function_ = distance.euclidean
            
        # Check that X and y have correct shape
        X, y = check_X_y(X, y)
        # Store the classes seen during fit
        self.classes_ = unique_labels(y)
        
        # Create a new empty dictionary into which we will store templates
        self.templates_ = dict()
        
        # Iterate through the classes creating a template for each and storing it
        for c in self.classes_:
            template = X[y == c].mean(axis = 0)
            self.templates_[c] = template
        
        # Return the classifier
        return self

    # The predict function to make a set of predictions for a set of query instances
    def predict(self, X):
        """Predict class labels of the input samples X.
        Parameters
        ----------
        X : array-like or sparse matrix of shape = [n_samples, n_features]
            The input samples. Internally, it will be converted to
            ``dtype=np.float32`` and if a sparse matrix is provided
            to a sparse ``csr_matrix``.
        Returns
        -------
        p : array of shape = [n_samples, ].
            The predicted class labels of the input samples. 
        """
        # Check is fit had been called by confirming that the teamplates_ dictiponary has been set up
        check_is_fitted(self, ['templates_'])

        # Check that the input features match the type and shape of the training features
        X = check_array(X)

        # Initialise an empty list to store the predictions made
        closest_classes = list()
        
        # Iterate through the query instances in the query dataset 
        for instance in X:
            # Insitialse best match to be the first template
            min_dist = self.distance_function_(instance, self.templates_[self.classes_[0]])
            closest_class = self.classes_[0]
            
            # Iterate through the templates to find the one closest to the query instance
            for c in self.templates_:
                dist = self.distance_function_(instance, self.templates_[c])
                if dist < min_dist:
                    min_dist = dist
                    closest_class = c
            closest_classes.append(closest_class)
            
        return np.array(closest_classes)

### Test the TemplateMatchClassifier

Do a simple test of the TemplateMatchClassifier

In [ ]:
a = np.array([[1,23,3,4], [5,6,7,8], [7,5,6,2], [4,9,12,43]])
y = np.array([1, 2, 2, 1])

In [ ]:
my_model = TemplateMatchClassifier(distance_metric='cosine')

In [ ]:
my_model.fit(a, y)

In [ ]:
my_model.templates_

In [ ]:
q = np.array([[2,15,6,21], [8,9,7,6]])

In [ ]:
my_model.predict(q)

Do simple Iris cross validation expeirment

In [ ]:
from sklearn.datasets import load_iris
clf = TemplateMatchClassifier()
iris = load_iris()
cross_val_score(clf, iris.data, iris.target, cv=10)

## Load & Partition Data

### Setup - IMPORTANT

Take only a sample of the dataset for fast testing

In [ ]:
data_sampling_rate = 0.1

Setup the number of folds for all grid searches (should be 5 - 10)

In [ ]:
cv_folds = 10

### Load & Partition Data

Load the dataset and explore it.

In [ ]:
dataset = pd.read_csv('fashion-mnist_train.csv')
dataset = dataset.sample(frac=data_sampling_rate) #take a sample from the dataset so everyhting runs smoothly
num_classes = 10
classes = {0: "T-shirt/top", 1:"Trouser", 2: "Pullover", 3:"Dress", 4:"Coat", 5:"Sandal", 6:"Shirt", 7:"Sneaker", 8:"Bag", 9:"Ankle boot"}
display(dataset.head())

Isolate the descriptive features we are interested in

In [ ]:
X = dataset[dataset.columns[1:]]
Y = np.array(dataset["label"])

In [ ]:
X = X/255

In [ ]:
X_train_plus_valid, X_test, y_train_plus_valid, y_test \
    = train_test_split(X, Y, random_state=0, \
                                    train_size = 0.7)

X_train, X_valid, y_train, y_valid \
    = train_test_split(X_train_plus_valid, \
                                        y_train_plus_valid, \
                                        random_state=0, \
                                        train_size = 0.5/0.7)

## Train and Evaluate a Simple Model

In [ ]:
my_model = TemplateMatchClassifier()
my_model.fit(X_train, y_train)

In [ ]:
# Make a set of predictions for the training data
y_pred = my_model.predict(X_train)

# Print performance details
accuracy = metrics.accuracy_score(y_train, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_train, y_pred))

# Print confusion matrix
# print(metrics.confusion_matrix(y_train, y_pred))

# Print nicer homemade confusion matrix
print("Confusion Matrix")
display(pd.crosstab(np.array(y_train), y_pred, rownames=['True'], colnames=['Predicted'], margins=True))

In [ ]:
# Make a set of predictions for the training data
y_pred = my_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
# print(metrics.confusion_matrix(y_train, y_pred))

# Print nicer homemade confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

## Do a Cross Validation Experiment With Our Model

In [ ]:
my_model = TemplateMatchClassifier()
scores = cross_val_score(my_model, X_train_plus_valid, y_train_plus_valid, cv=cv_folds, n_jobs=-1)
print(scores)

## Do a Grid Search Through Distance Metrics

In [ ]:
# Set up the parameter grid to seaerch
param_grid = [
 {'distance_metric': ['euclidean', 'cosine', 'manhattan']}
]

# Perform the search
my_tuned_model = GridSearchCV(TemplateMatchClassifier(), param_grid, cv=cv_folds, verbose = 2, n_jobs=-1)
my_tuned_model.fit(X_train_plus_valid, y_train_plus_valid)

# Print details
print("Best parameters set found on development set:")
print(my_tuned_model.best_params_)
print(my_tuned_model.best_score_)


In [ ]:
# Make a set of predictions for the test data
y_pred = my_tuned_model.predict(X_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(np.array(y_test), y_pred, rownames=['True'], colnames=['Predicted'], margins=True)